In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
import pprint
import matplotlib.pyplot as plt
from citipy import citipy


In [2]:
#Read data from csv file
airports_df = pd.read_csv("Airport_Output.csv", usecols = ["Airport Name","Airport Address", "Lat", "Lng"])

#Delete spaces on both sides of the column's name
airports_df.columns = airports_df.columns.str.replace(' ', '')

#Rename columns
airports_df=airports_df.rename(columns={"Lat": "Latitude", "Lng": "longitude"})

#Give format to the summary table 
#airports_df["Latitude"] = airports_df["Latitude"].map("{:.1f}".format)
#airports_df["longitude"] = airports_df["longitude"].map("{:,.1f}".format)
airports_df=airports_df.round({"Latitude": 0, "longitude": 0})

#Drop duplicates
airports_df= airports_df.drop_duplicates()

#Display Data
airports_df

Latitude  longitude                                        AirportName  \
0       41.0      -74.0               Newark Liberty International Airport   
1       34.0     -118.0                  Los Angeles International Airport   
2       42.0      -88.0                       O'Hare International Airport   
3       30.0      -95.0                           William P. Hobby Airport   
4       40.0      -75.0                 Philadelphia International Airport   
5       33.0     -112.0           Phoenix Sky Harbor International Airport   
6       29.0      -98.0                  San Antonio International Airport   
7       33.0     -117.0                    San Diego International Airport   
8       33.0      -97.0            Dallas/Fort Worth International Airport   
9       37.0     -122.0                San Francisco International Airport   
10      30.0      -98.0             Austin-Bergstrom International Airport   
11      30.0      -82.0                 Jacksonville International Airport   
12      38.0     -122.0                San Francisco International Airport   
13      40.0      -86.0                 Indianapolis International Airport   
14      40.0      -83.0          John Glenn Columbus International Airport   
16      35.0      -81.0            Charlotte Douglas International Airport   
17      48.0     -122.0               Seattle-Tacoma International Airport   
18      40.0     -105.0                       Denver International Airport   
19      32.0     -106.0                      El Paso International Airport   
20      42.0      -83.0          Detroit Metropolitan Wayne County Airport   
21      39.0      -77.0                       Dulles International Airport   
22      42.0      -71.0                 Boston Logan International Airport   
23      35.0      -90.0                      Memphis International Airport   
24      36.0      -87.0                    Nashville International Airport   
25      46.0     -123.0                     Portland International Airport   
26      35.0      -98.0                          Will Rogers World Airport   
27      36.0     -115.0                     McCarran International Airport   
28      39.0      -77.0  Baltimore/Washington International Thurgood Ma...   
29      38.0      -86.0                   Louisville International Airport   
30      43.0      -88.0             General Mitchell International Airport   
..       ...        ...                                                ...   
47      38.0      -97.0      Wichita Dwight D. Eisenhower National Airport   
48      30.0      -90.0  Louis Armstrong New Orleans International Airport   
50      41.0      -82.0            Cleveland Hopkins International Airport   
51      35.0     -119.0                              Meadows Field Airport   
52      28.0      -82.0                        Tampa International Airport   
54      21.0     -158.0                     Honolulu International Airport   
57      28.0      -97.0               Corpus Christi International Airport   
58      34.0     -117.0                   LA/Ontario International Airport   
59      39.0      -90.0            St. Louis Lambert International Airport   
60      38.0      -85.0                                 Blue Grass Airport   
61      38.0     -121.0                San Francisco International Airport   
62      40.0      -80.0                   Pittsburgh International Airport   
64      61.0     -150.0        Ted Stevens Anchorage International Airport   
65      39.0      -85.0  Cincinnati/Northern Kentucky International Air...   
67      36.0      -80.0               Piedmont Triad International Airport   
70      42.0      -84.0                             Toledo Express Airport   
71      41.0      -97.0                                    Lincoln Airport   
72      29.0      -81.0                      Orlando International Airport   
76      41.0      -85.0                   Fort Wayne International Airport   
77      43.0      -79.0   

In [3]:
types_df = pd.read_csv("ethnic_restr.csv")
types_df


ethnicity
0        chinese
1          cuban
2          czech
3         french
4         german
5          greek
6        haitian
7      hungarian
8         indian
9     indonesian
10         irish
11       israeli
12       italian
13      japanese
14        jewish
15        korean
16      lebanese
17       mexican
18  new american
19     pakistani
20        polish
21       russian
22  scandinavian
23      scottish
24     soul food
25       spanish
26          thai
27       turkish
28     ukrainian
29    vietnamese

In [4]:
# Import API key
gkey="h323GWkBwCy3RkzoJGx19eR-THnde-e6YJ4sAkN-zvZOyPCB35nTwT5EPzgpxzZmnvTkd-bWEPqY8TPyiUlTgiSnRXZI8rj-rJ63QOeUZAvHy5MxB-rvFoRkJ5K9XXYx"

# base url
base_url = "https://api.yelp.com/v3/businesses/search"

# set up api_key dictionary
headers = {
        'Authorization': 'Bearer %s' % gkey,
    }

# set up a parameters dictionary
def s_params(target_loc, target_key, target_type):
    params = {
    "location": target_loc, 
    "term": target_key,
    "categories": target_type,    
}
    return params
def s_query(base_url,headers,params):
    response = requests.request('GET', base_url, headers=headers, params=params).json()
    return response

In [5]:
loc = str(input("Where are you located?\n"))

name= []
city=[]
latitude= []
longitude= []
price= []
rating=[]
output=[]


def get_rest(cuisine,loc):
    
    #these are the additional params
    add_params = {"sort_by" : "distance","limit" : 1, "radius" : 1000} 

    #these are the original params
    params = s_params(loc,cuisine,"restaurants, All")

    #add additional params to our params from before
    s1_params = {**params,**add_params}

    s1_results = s_query(base_url,headers,s1_params)
    pprint.pprint(s1_results)
    
    if (s1_results):
        name = s1_results["businesses"][0]["name"]
        city = s1_results["businesses"][0]["location"]["city"]
        latitude = s1_results["businesses"][0]["coordinates"]["latitude"]
        longitude = s1_results["businesses"][0]["coordinates"]["longitude"]
        price = s1_results["businesses"][0]["price"]
        rating=s1_results["businesses"][0]["rating"]
        output = [name,city,latitude,longitude,price, rating] 
    else:
        output = ["NaN","NaN","NaN","NaN","NaN","NaN"]
    
    #print(f"The closest {cuisine.upper()} restaurant is {name} which has a rating of {rating} and is located at: \n {address} \n")   
    
    return output

#this is a long loop so print wait before starting
print("wait... ")

# use iterrows to iterate through pandas dataframe
# optional: use tqdm for progress bar: tqdm(iterator,total)
# like this: for index, row in tqdm(types_df.iterrows(), total=types_df.shape[0]):

for index, row in types_df.iterrows():

    # get restaurant type from df
    restr_type = str(row['ethnicity'])
   
    try:
        output = get_rest(restr_type, loc)
        name.append(output[0])
        city.append(output[1])
        latitude.append(output[2])
        longitude.append(output[3])
        price.append(output[4])
        rating.append(output[5])
        
        
    except (KeyError, IndexError):
        output = ["NaN","NaN","NaN","NaN","NaN","NaN"]
        name.append(output[0])
        city.append(output[1])
        latitude.append(output[2])
        longitude.append(output[3])
        price.append(output[4])
        rating.append(output[5])

#print done when done        
print("done!")

Where are you located?
2400 John Brantley Blvd, Morrisville
wait... 
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [{'alias': 'la-tapenade-wake',
                 'categories': [{'alias': 'mediterranean',
                                 'title': 'Mediterranean'}],
                 'coordinates': {'latitu

{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}},
 'total': 0}
{'businesses': [],
 'region': {'center': {'latitude': 35.876799798978816,
                       'longitude': -78.79257202148438}}

In [6]:
# Create a Dataframe
restaurants_df= pd.DataFrame({
                             "Restaurant": name,
                             "City": city,
                             "Latitude": latitude,
                             "longitude": longitude,                        
                             "price": price,
                             "rating": rating
                            }) 
#Delete spaces on both sides of the column's name
restaurants_df.columns = restaurants_df.columns.str.replace(' ', '')


#Convert data object as float
restaurants_df["Latitude"]=restaurants_df.Latitude.astype(float) 
restaurants_df["longitude"] = restaurants_df.longitude.astype(float)



#Delete Nan Values
restaurants_df.dropna()



Restaurant         City   Latitude  longitude price  \
5                       La Tapenade         Wake  35.874079 -78.790323     $   
12         California Pizza Kitchen      Raleigh  35.878299 -78.792665    $$   
17  Salsarita's Fresh Mexican Grill  Morrisville  35.873882 -78.790957    $$   

   rating  
5       3  
12      2  
17      3

In [7]:
#Give format to the summary table 
#restaurants_df["Latitude"] = restaurants_df["Latitude"].map("{:.1f}".format)
#restaurants_df["longitude"] = restaurants_df["longitude"].map("{:,.1f}".format)
restaurants_df=restaurants_df.round({"Latitude": 0, "longitude": 0})

In [8]:

#Save DataFrame as csv file
restaurants_df.to_csv( "restaurants_yelp.csv")                     


In [9]:
# Combine the data into a single dataset
combined_data = pd.merge(restaurants_df, airports_df, how="outer", on = ["Latitude", "longitude"])

# Display the data table for preview
combined_data.dropna()


Restaurant         City  Latitude  longitude price  \
27                      La Tapenade         Wake      36.0      -79.0     $   
28         California Pizza Kitchen      Raleigh      36.0      -79.0    $$   
29  Salsarita's Fresh Mexican Grill  Morrisville      36.0      -79.0    $$   

   rating                           AirportName  \
27      3  Raleigh-Durham International Airport   
28      2  Raleigh-Durham International Airport   
29      3  Raleigh-Durham International Airport   

                          AirportAddress  
27  2400 John Brantley Blvd, Morrisville  
28  2400 John Brantley Blvd, Morrisville  
29  2400 John Brantley Blvd, Morrisville

In [10]:
#Save DataFrame as csv file
pd.DataFrame.to_csv(restaurants_df, "restaurants_in _airports.csv")